In [1]:
import numpy as np
import pandas as pd
import scipy

In [2]:
df = pd.read_csv('data/freeman_well_4_eng.csv')

In [3]:
df.head()

,Depth,GR,Log_ILD,DT,RHOB,NPHI,PHI,PERM,velocity,GRI,vshale,PHIeff,formation_factor,swirr,permeability,Facies_code
0,7682.5,39.0321,0.9332,137.507,2.2382,0.5983,0.0188,0.002024,7272.357044,0.0,0.0,0.0188,3183.84791,1.261715,314.443398,0
1,7683.0,39.0321,0.9332,137.507,2.2382,0.5983,0.0188,0.002024,7272.357044,0.0,0.0,0.0188,3183.84791,1.261715,314.443398,0
2,7683.5,39.0321,0.9332,137.507,2.2382,0.5983,0.0188,0.002024,7272.357044,0.0,0.0,0.0188,3183.84791,1.261715,314.443398,0
3,7684.0,39.0321,0.9332,137.507,2.2382,0.5983,0.0188,0.002024,7272.357044,0.0,0.0,0.0188,3183.84791,1.261715,314.443398,0
4,7684.5,39.0321,0.9332,137.507,2.2382,0.5983,0.0188,0.002024,7272.357044,0.0,0.0,0.0188,3183.84791,1.261715,314.443398,0


In [4]:
from sklearn import preprocessing
from sklearn import utils
lab_enc = preprocessing.LabelEncoder()

In [5]:
df['Facies_code'].count()

7265

In [6]:
df['Facies']= lab_enc.fit_transform(df['Facies_code'])

In [7]:
df['Facies'].unique()

array([0, 1, 2], dtype=int64)

In [8]:
print(df.columns)

Index(['Depth', 'GR', 'Log_ILD', 'DT', 'RHOB', 'NPHI', 'PHI', 'PERM',
       'velocity', 'GRI', 'vshale', 'PHIeff', 'formation_factor', 'swirr',
       'permeability', 'Facies_code', 'Facies'],
      dtype='object')


In [9]:
# Define input features and target variable
X = df[['Depth', 'Log_ILD', 'NPHI','RHOB', 'vshale', 'PHIeff', 'swirr', 'Facies_code']]
y = df['PERM']

In [10]:
"""
from sklearn.feature_selection import SelectKBest, f_regression

# Select top 5 features based on F-value scores
selector = SelectKBest(f_regression, k=5)
X_new = selector.fit_transform(X, y)

# Get the indices of the selected features
selected_features = X.columns[selector.get_support(indices=True)].tolist()
"""

'\nfrom sklearn.feature_selection import SelectKBest, f_regression\n\n# Select top 5 features based on F-value scores\nselector = SelectKBest(f_regression, k=5)\nX_new = selector.fit_transform(X, y)\n\n# Get the indices of the selected features\nselected_features = X.columns[selector.get_support(indices=True)].tolist()\n'

In [11]:
## selected_features

In [12]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import joblib

In [21]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Define dictionary of models and their hyperparameters
models = {
    'svm': {
        'model': make_pipeline(StandardScaler(), SVR()),
        'params': {
            'svr__kernel': ['linear', 'rbf', 'poly'],
            'svr__C': [0.1, 1, 10],
            'svr__gamma': ['scale', 'auto']
        }
    },
    'random_forest': {
        'model': RandomForestRegressor(),
        'params': {
            'n_estimators': [50, 100, 150],
            'max_depth': [10, 20, 30]
        }
    },
    'decision_tree': {
        'model': DecisionTreeRegressor(),
        'params': {
            'max_depth': [5, 10, 15],
            'min_samples_split': [2, 5, 10]
        }
    },
    'neural_network': {
        'model': MLPRegressor(),
        'params': {
            'hidden_layer_sizes': [(10,), (20,), (30,)],
            'activation': ['relu', 'tanh', 'logistic'],
            'max_iter': [500, 1000]
        }
    },
    'gradient_boosting': {
        'model': GradientBoostingRegressor(),
        'params': {
            'learning_rate': [0.05, 0.1, 0.2],
            'n_estimators': [50, 100, 150],
            'max_depth': [3, 5, 7]
        }
    }
}

In [22]:
scores = []
# Loop over each model and perform grid search with cross-validation to find best hyperparameters 
## scoring='neg_mean_squared_error'
for model_name, model in models.items():
    clf = GridSearchCV(model['model'], model['params'], cv=5, n_jobs=-1, return_train_score=False)
    clf.fit(X_train, y_train)
    best_params = clf.best_params_
    
    # Evaluate best model on test set
    y_pred = clf.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = clf.score(X_test, y_test)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': best_params,
        'RMSE': rmse,
        'R-squared': r2
    })
    
    # Save the best model for each method
    joblib.dump(clf.best_estimator_, f"models/{model_name}_best_model.pkl")

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


In [23]:
best_model_df = pd.DataFrame(scores, columns=['model', 'best_params', 'best_score', 'RMSE', 'R-squared'])
best_model_df

,model,best_params,best_score,RMSE,R-squared
0,svm,"{'svr__C': 10, 'svr__gamma': 'scale', 'svr__ke...",0.143760,306.821062,0.324236
1,random_forest,"{'max_depth': 10, 'n_estimators': 50}",0.750174,90.489694,0.941221
2,decision_tree,"{'max_depth': 10, 'min_samples_split': 2}",0.784335,80.988583,0.952916
3,neural_network,"{'activation': 'logistic', 'hidden_layer_sizes...",-0.000654,373.248200,-0.000047
4,gradient_boosting,"{'learning_rate': 0.2, 'max_depth': 3, 'n_esti...",0.788130,144.809411,0.849472


In [16]:
best_model_df.to_csv('data/best_model.csv', index=False)

In [17]:
# Load the models from the model folder
model_rf = joblib.load('models/random_forest_best_model.pkl')
model_dt = joblib.load('models/decision_tree_best_model.pkl')
model_gb = joblib.load('models/gradient_boosting_best_model.pkl')

In [18]:
# Predict y values for x_test
y_pred_rf = model_rf.predict(X_test)
y_pred_dt = model_dt.predict(X_test)
y_pred_gb = model_gb.predict(X_test)

X_test_df = pd.DataFrame(X_test, columns=['Depth', 'Log_ILD', 'NPHI','RHOB', 'vshale', 'PHIeff', 'swirr', 'Facies_code'])

# Create a dataframe with x_test, y_test, and y_predict
results_df = pd.DataFrame({'Depth': X_test_df['Depth'],
                           'Log_ILD': X_test_df['Log_ILD'],
                           'NPHI': X_test_df['NPHI'],
                           'RHOB': X_test_df['RHOB'],
                           'vshale': X_test_df['vshale'],
                           'swirr': X_test_df['swirr'],
                           'Facies_code': X_test_df['Facies_code'],
                           'effective porosity': X_test_df['PHIeff'],
                           'Actual Permeability': y_test,
                           'rf_Permeability': y_pred_rf,
                           'dt_Permeability': y_pred_dt,
                           'gb_Permeability': y_pred_gb})

# Print the dataframe
results_df.to_csv('results_test.csv', index=False)
results_df.head()

,Depth,Log_ILD,NPHI,RHOB,vshale,swirr,Facies_code,effective porosity,Actual Permeability,rf_Permeability,dt_Permeability,gb_Permeability
742,8056.5,0.7639,0.5036,2.2186,0.327503,0.076724,2,0.210415,103.702727,102.897186,100.438528,104.028909
4739,10076.0,1.4776,0.3740,2.3766,0.460739,0.136401,2,0.087161,0.809119,0.809343,0.799591,0.850160
6435,10927.5,1.2484,0.3803,2.4116,0.302207,0.163782,2,0.085104,0.276733,0.276761,0.248382,0.555255
6602,11011.0,0.7850,0.3888,2.3935,0.253509,0.148363,2,0.103730,0.483099,0.483054,0.248382,0.602359
7166,11308.0,0.8590,0.5511,2.3986,0.995671,0.152405,2,0.000880,0.413088,0.413088,0.248382,0.555255


In [20]:
# Predict y values for x_train
y_pred_rf_train = model_rf.predict(X_train)
y_pred_dt_train = model_dt.predict(X_train)
y_pred_gb_train = model_gb.predict(X_train)

X_train_df = pd.DataFrame(X_train, columns=['Depth', 'Log_ILD', 'NPHI','RHOB', 'vshale', 'PHIeff', 'swirr', 'Facies_code'])

# Create a dataframe with x_train, y_train, and y_predict
train_df = pd.DataFrame({'Depth': X_train_df['Depth'],
                           'Log_ILD': X_train_df['Log_ILD'],
                           'NPHI': X_train_df['NPHI'],
                           'RHOB': X_train_df['RHOB'],
                           'vshale': X_train_df['vshale'],
                           'swirr': X_train_df['swirr'],
                           'Facies_code': X_train_df['Facies_code'],
                           'effective porosity': X_train_df['PHIeff'],
                           'Actual Permeability': y_train,
                           'rf_Permeability': y_pred_rf_train,
                           'dt_Permeability': y_pred_dt_train,
                           'gb_Permeability': y_pred_gb_train})

# Print the dataframe
train_df.to_csv('results.csv', index=False)
train_df.head()

,Depth,Log_ILD,NPHI,RHOB,vshale,swirr,Facies_code,effective porosity,Actual Permeability,rf_Permeability,dt_Permeability,gb_Permeability
3824,9613.5,1.3141,0.3588,2.4017,0.364905,0.155009,2,0.083303,0.375014,0.375153,0.248382,0.555255
5753,10583.0,1.8406,0.3862,2.3942,0.267065,0.148945,2,0.101413,0.472103,0.472103,0.248382,0.602359
3967,9689.0,2.1227,0.3778,2.3616,0.269407,0.127196,2,0.122799,1.282305,1.282305,1.680916,1.287773
6570,10995.0,0.7959,0.4434,2.3858,0.416998,0.142675,2,0.086922,0.610977,0.611260,0.799591,0.659817
987,8179.0,0.9905,0.4988,2.2684,0.248710,0.089176,2,0.187773,22.483303,22.479362,22.855801,23.577823


In [27]:
print(train_df.columns)

Index(['Depth', 'Log_ILD', 'NPHI', 'RHOB', 'vshale', 'swirr', 'Facies_code',
       'effective porosity', 'Actual Permeability', 'rf_Permeability',
       'dt_Permeability', 'gb_Permeability'],
      dtype='object')
